In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import re
import random
import warnings
import tensorflow as tf

from tensorflow import keras as ks
from tensorflow.keras import layers
from keras.models import load_model

In [ ]:
def clean_data(text):
    text = text.lower()
    text = re.sub(r'[^(a-zA-Z0-9)\s\*\+-\/\(\)=&|]','', text)
    return text

train = pd.read_csv("/kaggle/input/60k-stack-overflow-questions-with-quality-rate/train.csv")
valid = pd.read_csv("/kaggle/input/60k-stack-overflow-questions-with-quality-rate/valid.csv")

maxlength = 75

train['Body'] = train['Body'].apply(clean_data)
valid['Body'] = valid['Body'].apply(clean_data)

train2=train.copy()
    
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=10000, filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n\r', lower=True, split=" ")
tokenizer.fit_on_texts(train['Body'])
train_x = ks.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences(train['Body']), maxlen=maxlength)
valid_x = ks.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences(valid['Body']), maxlen=maxlength)

train_y = train['Y'].map({'LQ_CLOSE':0, 'LQ_EDIT':1, 'HQ':2})
valid_y = valid['Y'].map({'LQ_CLOSE':0, 'LQ_EDIT':1, 'HQ':2})

train_y = ks.utils.to_categorical(train_y, num_classes=3)
valid_y = ks.utils.to_categorical(valid_y, num_classes=3)

In [ ]:
print(tokenizer.word_index)

In [ ]:
model = ks.Sequential()
model.add(layers.Embedding(input_length=maxlength, input_dim=10000, output_dim=128))    
model.add(layers.LSTM(64, return_sequences=True))
model.add(layers.LSTM(64))
model.add(layers.Dense(3, activation="softmax"))
model.compile(optimizer=ks.optimizers.SGD(learning_rate=(0.55)), loss=ks.losses.CategoricalCrossentropy(), metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(train_x, train_y, batch_size=128, epochs=25)
loss, acc = model.evaluate(valid_x, valid_y, verbose=1)
print('Loss:\t\t', loss, '\nAccuracy:\t', acc)

In [ ]:
model.save("rnn")